In [257]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
from datetime import datetime

In [258]:
import numpy as np
import pandas as pd

In [259]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [298]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData, Table, Column, ForeignKey, Integer, String, Float, DateTime, inspect, distinct, desc, and_

In [261]:
# Create Database Connection
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base.metadata.create_all(engine)

In [262]:
# reflect an existing database into a new model
Base=automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# save refernece to the table
Measurement = Base.classes.measurement
Station = Base.classes.station

session = Session(engine)

In [263]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [264]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [265]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

In [266]:
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [267]:
engine.execute('SELECT * FROM measurement LIMIT 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0)]

In [268]:
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [269]:
engine.execute('SELECT * FROM station LIMIT 5').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

# Exploratory Climate Analysis

In [270]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database
end_date, = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
begin_date=dt.datetime.strptime(end_date, '%Y-%m-%d')-dt.timedelta(days=365)
end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')
print(end_date,begin_date)

# Perform a query to retrieve the data and precipitation scores
# data = session.query(Measurement.id,Measurement.station,Measurement.date, Measurement.prcp, Measurement.tobs)\
# .filter(and_(Measurement.date>=begin_date, Measurement.date<=end_date)).all()
data = session.query(Measurement.id,Measurement.station,Measurement.date, Measurement.prcp, Measurement.tobs)\
.filter(Measurement.date>=begin_date).filter(Measurement.date<=end_date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
# Sort the dataframe by date
prcp_data = pd.DataFrame(data).set_index('date').sort_values(by='date', ascending=False)

# Use Pandas Plotting with Matplotlib to plot the data
prcp_data

2017-08-23 00:00:00 2016-08-23 00:00:00


,id,station,prcp,tobs
date,,,,
2017-08-23,19550,USC00516128,0.45,76.0
2017-08-23,12187,USC00519523,0.08,82.0
2017-08-23,7635,USC00514830,0.00,82.0
2017-08-23,2724,USC00519397,0.00,81.0
2017-08-22,12186,USC00519523,0.00,82.0
...,...,...,...,...
2016-08-24,11852,USC00519523,1.22,79.0
2016-08-24,19203,USC00516128,1.45,74.0
2016-08-24,5092,USC00513117,2.15,76.0


In [271]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_data["prcp"].agg(["mean","median", "sum", "count", "max", "min", "std", "var"])

mean         0.176462
median       0.020000
sum        355.570000
count     2015.000000
max          6.700000
min          0.000000
std          0.460288
var          0.211865
Name: prcp, dtype: float64

In [272]:
# Design a query to show how many stations are available in this dataset.
stations_lastyr = prcp_data.station.nunique()
stations, = session.query(func.count(distinct(Measurement.station))).order_by
(f'There are {stations_lastyr} unique weather stations with measurments taken in the last year of data. There are {stations} unique weather stations in the entire dataset.')

'There are 7 unique weather stations with measurments taken in the last year of data. There are 9 unique weather stations in the entire dataset.'

In [404]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
active_all = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station) \
.order_by(desc(func.count(Measurement.station))).all()
active = prcp_data["station"].value_counts() #Returns descending by default
active = pd.DataFrame(active)
# prcp_data["station"].value_counts(normalize=True) #returns percentages of whole instead of count!
print('This is the dataset filtered for the last year of data.')
active

This is the dataset filtered for the last year of data.


,station
USC00519397,360
USC00519281,351
USC00516128,348
USC00513117,342
USC00519523,336
USC00514830,304
USC00517948,182


In [405]:
print('This is the whole dataset.')
active_all = [[ i for i, j in active_all ], 
       [ j for i, j in active_all ]] 

active_all

This is the whole dataset.


[['USC00519281',
  'USC00519397',
  'USC00513117',
  'USC00519523',
  'USC00516128',
  'USC00514830',
  'USC00511918',
  'USC00517948',
  'USC00518838'],
 [2772, 2724, 2709, 2669, 2612, 2202, 1979, 1372, 511]]

In [396]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
most_active = active.index[0]
active_agg = prcp_data.loc[prcp_data["station"] == most_active]
active_agg["tobs"].agg(["mean", "max", "min"])

KeyError: (1, 1)

In [ ]:
most_active_all = 

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`


In [65]:
# class Measurement_two(Base):
#     __tablename__= "measurement"
#     id = Column(Integer, primary_key=True)
#     station = Column(String(200))
#     date = Column(DateTime)
#     prcp = Column(Float)
#     tobs = Column(Float)

# class Station_two(Base):
#     __tablename__= "station"
#     id = Column(Integer, primary_key=True)
#     station = Column(String(200))
#     name = Column(String(200))
#     latitude = Column(Float)
#     longitude = Column(Float)
#     elevation = Column(Float)